**Jupyter notebook avaialble @ [www.github.com/iaja/tf-guru/dataset/2017-11-11-TFRecord.ipynb](www.github.com/iaja/tf-guru/dataset/2017-11-11-TFRecord-DatasetHandling.ipynb)**

In [16]:
import tensorflow as tf
import numpy as np
import tempfile

In [17]:
tf.reset_default_graph()
sess = tf.Session()

### Preprocessing Data: [Use tf.SequenceExample](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/example/example.proto)

```
Data ---> Preprocessing ---> TFRecords ---> Input Graph ---> Model
```

RNNs are used for sequential data that has inputs and/or outputs at multiple time steps. Tensorflow comes with a protocol buffer definition to deal with such data: `tf.SequenceExample`.


You can load data directly from your Python/Numpy arrays, but it’s probably in your best interest to use tf.SequenceExample instead. This data structure consists of a “context” for non-sequential features and “feature_lists” for sequential features. It’s somewhat verbose (it blew up my latest dataset by 10x), but it comes with a few benefits that are worth it:

- **Easy distributed training.** Split up data into multiple TFRecord files, each containing many SequenceExamples, and use Tensorflow’s built-in support for distributed training.
- **Reusability.** Other people can re-use your model by bringing their own data into tf.SequenceExample format. No model code changes required.
- **Use of Tensorflow data loading pipelines functions like tf.parse_single_sequence_example.** Libraries like tf.learn also come with convenience function that expect you to feed data in protocol buffer format.
- **Separation of data preprocessing and model code.** Using tf.SequenceExample forces you to separate your data preprocessing and Tensorflow model code. This is good practice, as your model shouldn’t make any assumptions about the input data it gets.


In practice, you write a little script that converts your data into tf.SequenceExample format and then writes one or more TFRecord files. These TFRecord files are parsed by Tensorflow to become the input to your model:

- Convert your data into tf.SequenceExample format
- Write one or more TFRecord files with the serialized data
- Use tf.TFRecordReader to read examples from the file
- Parse each example using tf.parse_single_sequence_example (Not in the official docs yet)

In [18]:

sequences = [[1, 2, 3], [4, 5, 1], [1, 2]]
label_sequences = [[0, 1, 0], [1, 0, 0], [1, 1]]

def make_example(sequence, labels):
    # The object we return
    example_sequence = tf.train.SequenceExample()
    # A non-sequential feature of our example
    sequence_length = len(sequence)
    example_sequence.context.feature["length"].int64_list.value.append(sequence_length)
    # Feature lists for the two sequential features of our example
    fl_tokens = example_sequence.feature_lists.feature_list["tokens"]
    fl_labels = example_sequence.feature_lists.feature_list["labels"]
    for token, label in zip(sequence, labels):
        fl_tokens.feature.add().int64_list.value.append(token)
        fl_labels.feature.add().int64_list.value.append(label)
    return example_sequence



In [19]:
# A single serialized example
# (You can read this from a file using TFRecordReader)
example_sequence = make_example([1, 2, 3], [0, 1, 0]).SerializeToString()

# Define how to parse the example
context_features = {
    "length": tf.FixedLenFeature([], dtype=tf.int64)
}
sequence_features = {
    "tokens": tf.FixedLenSequenceFeature([], dtype=tf.int64), #tf.VarLenFeature(tf.int64),
    "labels": tf.FixedLenSequenceFeature([], dtype=tf.int64)
}


In [20]:
# Parse the example (returns a dictionary of tensors)
context_parsed, sequence_parsed = tf.parse_single_sequence_example(
    serialized=example_sequence,
    context_features=context_features,
    sequence_features=sequence_features
)

In [21]:
sess.run(context_parsed)
sess.run(sequence_parsed)

{'labels': array([0, 1, 0]), 'tokens': array([1, 2, 3])}

### Write all examples into a TFRecords file

In [22]:
def save_into_tfrecord(sequences, label_sequences):
    with tempfile.NamedTemporaryFile() as fp:
        
        writer = tf.python_io.TFRecordWriter(fp.name)
        #get each sequence along with its labels and write it to a file as Sequence Example.
        for sequence, label_sequence in zip(sequences, label_sequences):
            example_sequence = make_example(sequence, label_sequence)
            writer.write(example_sequence.SerializeToString())
        
        writer.close()
        print("Wrote to {}".format(fp.name))
        return fp.name

In [23]:
filename = save_into_tfrecord(sequences, label_sequences)
filename

Wrote to /tmp/tmpznwjka42


'/tmp/tmpznwjka42'

In [25]:
def read_and_decode_single_example(filename):

    filename_queue = tf.train.string_input_producer([filename],
                                                num_epochs=None)

    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)

    # Define how to parse the example
    context_features = {
        "length": tf.FixedLenFeature([], dtype=tf.int64)
    }
    sequence_features = {
        "tokens": tf.FixedLenSequenceFeature([], dtype=tf.int64), #tf.VarLenFeature(tf.int64),
        "labels": tf.FixedLenSequenceFeature([], dtype=tf.int64)
    }


    return serialized_example, context_features, sequence_features

example_sequence,context_features,sequence_features = read_and_decode_single_example(filename)

In [26]:
# Parse the example (returns a dictionary of tensors)
context_parsed, sequence_parsed = tf.parse_single_sequence_example(
    serialized=example_sequence,
    context_features=context_features,
    sequence_features=sequence_features
)

In [ ]:
sess.run(context_parsed)
sess.run(sequence_parsed)

# Convert this notebook for Docs

In [9]:
! jupyter nbconvert --to markdown --output-dir ../docs/_posts 2017-11-11-TFRecord.ipynb

[NbConvertApp] Converting notebook 2017-11-11-TFRecord.ipynb to markdown
[NbConvertApp] Writing 6351 bytes to ../docs/_posts/2017-11-11-TFRecord.md
